In [7]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook



In [1]:
import json
with open('C:\\Users\\Sly\\.secret\\yelp_api.json') as f: #change the path to match YOUR path!!
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [2]:
from yelpapi import YelpAPI
yelp_api = YelpAPI(login["api-key"], timeout_s=5.0)
yelp_api


In [3]:
# Quick Test Query
results = yelp_api.search_query(location='Chicago, IL',
                                       term='Pizza')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [5]:
# set our API call parameters 
LOCATION = 'Chicago,IL'
TERM = 'Pizza'


In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_chicago_pizza.json"
JSON_FILE



'Data/results_in_progress_chicago_pizza.json'

In [8]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_chicago_pizza.json not found. Saving empty list to file.


In [11]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [12]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [13]:
## How many results total?
total_results = results['total']
total_results



5300

In [14]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page



20

In [15]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages



265

In [16]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [17]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  



In [18]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)



  0%|          | 0/265 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [19]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,DXwSYgiXqIVNdO9dazel6w,pequods-pizzeria-chicago,Pequod's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/8QJUNb...,False,https://www.yelp.com/biz/pequods-pizzeria-chic...,7688,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.92187, 'longitude': -87.664486}","[pickup, delivery]",$$,"{'address1': '2207 N Clybourn Ave', 'address2'...",+17733271512,(773) 327-1512,2158.708458
1,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/abCXuj...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,7167,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[pickup, delivery]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,4000.999053
2,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3585,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88513231411071, 'longitude': -...",[delivery],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13126161200,(312) 616-1200,4994.255603
3,TBoYnkPNFn9ASccYCYsJfw,nancys-pizza-west-loop-chicago,Nancy's Pizza West Loop,https://s3-media3.fl.yelpcdn.com/bphoto/Ew6IzX...,False,https://www.yelp.com/biz/nancys-pizza-west-loo...,166,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 41.88331, 'longitude': -87.65269}","[pickup, delivery, restaurant_reservation]",$$,"{'address1': '1000 W Washington Blvd', 'addres...",+13127339920,(312) 733-9920,3179.050691
4,uTkMnmsQ1-iOtbyFmW0uIw,professor-pizza-chicago,Professor Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/QzMH67...,False,https://www.yelp.com/biz/professor-pizza-chica...,30,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.889217, 'longitude': -87.651297}","[delivery, pickup]",NaN,"{'address1': '406 N Sangamon St', 'address2': ...",+13126615319,(312) 661-5319,2822.252327


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,pkc8oSE8lSEGFvXla1J_Vg,la-pizza-and-la-pasta-at-eataly-chicago-chicago,La Pizza & La Pasta at Eataly Chicago,https://s3-media2.fl.yelpcdn.com/bphoto/9gkSDq...,False,https://www.yelp.com/biz/la-pizza-and-la-pasta...,170,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 41.892189, 'longitude': -87.626031}",[delivery],$$,"{'address1': '43 E Ohio St', 'address2': '', '...",+13125218700,(312) 521-8700,4518.138331
996,nhSsJ9ABrlt11G_1-RrqDw,el-ranchito-restaurant-chicago-5,El Ranchito Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/NQzoWs...,False,https://www.yelp.com/biz/el-ranchito-restauran...,461,"[{'alias': 'mexican', 'title': 'Mexican'}, {'a...",2.0,"{'latitude': 41.93276, 'longitude': -87.7129}","[delivery, pickup]",$,"{'address1': '2829 N Milwaukee Ave', 'address2...",+17732271688,(773) 227-1688,4227.722420
997,MVrDteOcDumw3RUnVvRAxw,dreams-pizza-on-the-9-markham,Dream's Pizza On The 9,https://s3-media1.fl.yelpcdn.com/bphoto/9R19Fr...,False,https://www.yelp.com/biz/dreams-pizza-on-the-9...,22,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 41.60239, 'longitude': -87.71635}","[delivery, pickup]",NaN,"{'address1': '15743 Crawford Ave', 'address2':...",+17089860700,(708) 986-0700,33810.331588
998,Dt6PM-PdUrGBN9ss1U6rtg,d-agostino-s-pizza-and-pub-river-grove-river-g...,D’Agostino’s Pizza and Pub River Grove,https://s3-media4.fl.yelpcdn.com/bphoto/9fgaqS...,False,https://www.yelp.com/biz/d-agostino-s-pizza-an...,39,"[{'alias': 'pubs', 'title': 'Pubs'}, {'alias':...",3.0,"{'latitude': 41.936528, 'longitude': -87.835631}","[delivery, pickup]",NaN,"{'address1': '3131 Thatcher Ave', 'address2': ...",+17084533247,(708) 453-3247,13517.016481
999,2tErhaqgdXstrilg3I0Jgw,the-map-room-chicago,The Map Room,https://s3-media1.fl.yelpcdn.com/bphoto/-AO6aW...,False,https://www.yelp.com/biz/the-map-room-chicago?...,804,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 41.9176, 'longitude': -87.67979}",[delivery],$$,"{'address1': '1949 N Hoyne Ave', 'address2': '...",+17732529351,(773) 252-9351,1394.161172


In [20]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()


0

In [21]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NY_pizza.csv.gz', compression='gzip',index=False)

